In [ ]:
import tkinter as tk
import pandas as pd
import random

# 初始化玩家資料
scores = pd.DataFrame({'Player': ['Player 1', 'Player 2'],'Score': [0, 0]})

# 分數更新函式
def update_scores():
    score_label_1.config(text=f"Player 1: {scores.loc[0, 'Score']}")
    score_label_2.config(text=f"Player 2: {scores.loc[1, 'Score']}")
# 每個小遊戲勝者後加分
def end_game(winner):
    if winner == 1:
        scores.loc[0, 'Score'] += 1
    elif winner == 2:
        scores.loc[1, 'Score'] += 1
    update_scores()

# 遊戲1
def game_minesweeper():
    new_window = tk.Toplevel(root)
    new_window.title("1-100 踩地雷遊戲")

    secret = random.randint(1, 100)
    low = 1
    high = 100
    current_player = [1]

    status_label = tk.Label(new_window, text=f"目前猜測範圍：{low} 到 {high}")
    status_label.pack(pady=5)

    player_label = tk.Label(new_window, text=f"玩家 {current_player[0]} 請猜數字")
    player_label.pack(pady=5)

    entry = tk.Entry(new_window)
    entry.pack(pady=5)

    def check_guess():
        nonlocal low, high, secret

        try:
            guess = int(entry.get())
        except ValueError:
            result_label.config(text="請輸入有效的整數！")
            return

        if guess <= low or guess >= high:
            result_label.config(text=f"猜測超出範圍！請輸入 {low + 1} 到 {high - 1} 之間的數字")
            return

        if guess == secret:
            loser = current_player[0]
            winner = 2 if loser == 1 else 1
            result_label.config(text=f"猜中了！地雷是 {secret}，玩家 {loser} 輸了")
            end_game(winner)
            tk.Button(new_window, text="關閉", command=new_window.destroy).pack(pady=10)
            return
        elif guess < secret:
            low = guess
        else:
            high = guess

        current_player[0] = 2 if current_player[0] == 1 else 1
        player_label.config(text=f"玩家 {current_player[0]} 請猜數字")
        status_label.config(text=f"目前猜測範圍：{low} 到 {high}")
        result_label.config(text="繼續猜！")
        entry.delete(0, tk.END)

    tk.Button(new_window, text="送出猜測", command=check_guess).pack(pady=5)

    result_label = tk.Label(new_window, text="")
    result_label.pack(pady=5)

# 遊戲2
def game_yahtzee():
    def roll_dice():
        return [random.randint(1, 6) for _ in range(5)]

    def display_dice(dice):
        return ' '.join(str(d) for d in dice)

    def evaluate_hand(dice):
        counts = {x: dice.count(x) for x in set(dice)}
        values = list(counts.values())
        total = sum(dice)

        if 5 in values:
            return ("五個相同", 6, total)
        elif 4 in values:
            return ("四個相同", 5, total)
        elif sorted(values) == [2, 3]:
            return ("葫蘆", 4, total)
        elif sorted(dice) in [list(range(1, 6)), list(range(2, 7))]:
            return ("順子", 3, total)
        elif 3 in values:
            return ("三個相同", 2, total)
        elif values.count(2) == 2:
            return ("兩對", 1, total)
        else:
            return ("雜牌", 0, total)

    def reroll_dice(dice, name):
        while True:
            keep = simpledialog.askstring("保留骰子", f"{name} 請輸入要保留的骰子位置 (1~5，用空格分隔)：\n目前骰子：{display_dice(dice)}")
            if not keep:
                return roll_dice()
            try:
                keep_indices = [int(i) - 1 for i in keep.split() if 1 <= int(i) <= 5]
                new_dice = [dice[i] if i in keep_indices else random.randint(1, 6) for i in range(5)]
                return new_dice
            except:
                continue

    import tkinter.simpledialog as simpledialog

    name1 = "Player 1"
    name2 = "Player 2"

    dice1 = roll_dice()
    dice2 = roll_dice()

    dice1_final = reroll_dice(dice1, name1)
    dice2_final = reroll_dice(dice2, name2)

    hand1, rank1, total1 = evaluate_hand(dice1_final)
    hand2, rank2, total2 = evaluate_hand(dice2_final)

    result = tk.Toplevel(root)
    result.title("簡化 Yahtzee 結果")

    tk.Label(result, text=f"{name1} 骰子：{display_dice(dice1_final)} → {hand1}（{total1}分）").pack(pady=3)
    tk.Label(result, text=f"{name2} 骰子：{display_dice(dice2_final)} → {hand2}（{total2}分）").pack(pady=3)

    outcome = ""
    if rank1 > rank2:
        outcome = f"🎉 {name1} 勝利！（{hand1} 勝 {hand2}）"
        end_game(1)
    elif rank2 > rank1:
        outcome = f"🎉 {name2} 勝利！（{hand2} 勝 {hand1}）"
        end_game(2)
    else:
        if total1 > total2:
            outcome = f"🎉 {name1} 勝利！（同牌型，總點數較高）"
            end_game(1)
        elif total2 > total1:
            outcome = f"🎉 {name2} 勝利！（同牌型，總點數較高）"
            end_game(2)
        else:
            outcome = "🤝 平手！（牌型與點數皆相同）"

    tk.Label(result, text=outcome, font=("Arial", 12, "bold"), fg="blue").pack(pady=10)
    tk.Button(result, text="關閉", command=result.destroy).pack(pady=5)


#遊戲3
def game_tetris():
    import pygame
    import random

    colors = [
        (0, 0, 0),
        (120, 37, 179),
        (100, 179, 179),
        (80, 34, 22),
        (80, 134, 22),
        (180, 34, 22),
        (180, 34, 122),
    ]

    class Figure:
        x = 0
        y = 0
        figures = [
            [[1, 5, 9, 13], [4, 5, 6, 7]],
            [[4, 5, 9, 10], [2, 6, 5, 9]],
            [[6, 7, 9, 10], [1, 5, 6, 10]],
            [[1, 2, 5, 9], [0, 4, 5, 6], [1, 5, 9, 8], [4, 5, 6, 10]],
            [[1, 2, 6, 10], [5, 6, 7, 9], [2, 6, 10, 11], [3, 5, 6, 7]],
            [[1, 4, 5, 6], [1, 4, 5, 9], [4, 5, 6, 9], [1, 5, 6, 9]],
            [[1, 2, 5, 6]],
        ]

        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.type = random.randint(0, len(self.figures) - 1)
            self.color = random.randint(1, len(colors) - 1)
            self.rotation = 0

        def image(self):
            return self.figures[self.type][self.rotation]

        def rotate(self):
            self.rotation = (self.rotation + 1) % len(self.figures[self.type])

    class Tetris:
        level = 2
        score = 0
        state = "start"
        field = []
        height = 0
        width = 0
        x = 100
        y = 60
        zoom = 20
        figure = None

        def __init__(self, height, width):
            self.height = height
            self.width = width
            self.field = [[0 for _ in range(width)] for _ in range(height)]
            self.new_figure()

        def new_figure(self):
            self.figure = Figure(3, 0)

        def intersects(self):
            for i in range(4):
                for j in range(4):
                    if i * 4 + j in self.figure.image():
                        x = j + self.figure.x
                        y = i + self.figure.y
                        if x < 0 or x >= self.width or y >= self.height:
                            return True
                        if y >= 0 and self.field[y][x] > 0:
                            return True
            return False

        def break_lines(self):
            lines = 0
            for i in range(self.height - 1, -1, -1):
                if 0 not in self.field[i]:
                    del self.field[i]
                    self.field.insert(0, [0 for _ in range(self.width)])
                    lines += 1
            self.score += lines ** 2

        def go_space(self):
            while not self.intersects():
                self.figure.y += 1
            self.figure.y -= 1
            self.freeze()

        def go_down(self):
            self.figure.y += 1
            if self.intersects():
                self.figure.y -= 1
                self.freeze()

        def freeze(self):
            for i in range(4):
                for j in range(4):
                    if i * 4 + j in self.figure.image():
                        self.field[i + self.figure.y][j + self.figure.x] = self.figure.color
            self.break_lines()
            self.new_figure()
            if self.intersects():
                self.state = "gameover"

        def go_side(self, dx):
            self.figure.x += dx
            if self.intersects():
                self.figure.x -= dx

        def rotate(self):
            old_rotation = self.figure.rotation
            self.figure.rotate()
            if self.intersects():
                self.figure.rotation = old_rotation

    pygame.init()
    screen = pygame.display.set_mode((400, 500))
    pygame.display.set_caption("俄羅斯方塊")
    clock = pygame.time.Clock()
    game = Tetris(20, 10)
    fps = 25
    counter = 0
    pressing_down = False
    done = False

    while not done:
        if game.figure is None:
            game.new_figure()
        counter += 1
        if counter > 100000:
            counter = 0

        if counter % (fps // game.level // 2) == 0 or pressing_down:
            if game.state == "start":
                game.go_down()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    game.rotate()
                elif event.key == pygame.K_DOWN:
                    pressing_down = True
                elif event.key == pygame.K_LEFT:
                    game.go_side(-1)
                elif event.key == pygame.K_RIGHT:
                    game.go_side(1)
                elif event.key == pygame.K_SPACE:
                    game.go_space()
                elif event.key == pygame.K_ESCAPE:
                    game.__init__(20, 10)
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_DOWN:
                    pressing_down = False

        screen.fill((255, 255, 255))

        for i in range(game.height):
            for j in range(game.width):
                pygame.draw.rect(screen, (128, 128, 128),
                                 [game.x + game.zoom * j, game.y + game.zoom * i, game.zoom, game.zoom], 1)
                if game.field[i][j] > 0:
                    pygame.draw.rect(screen, colors[game.field[i][j]],
                                     [game.x + game.zoom * j + 1, game.y + game.zoom * i + 1, game.zoom - 2,
                                      game.zoom - 2])

        if game.figure is not None:
            for i in range(4):
                for j in range(4):
                    if i * 4 + j in game.figure.image():
                        pygame.draw.rect(screen, colors[game.figure.color],
                                         [game.x + game.zoom * (j + game.figure.x) + 1,
                                          game.y + game.zoom * (i + game.figure.y) + 1,
                                          game.zoom - 2, game.zoom - 2])

        font = pygame.font.SysFont('Calibri', 25, True, False)
        text = font.render("Score: " + str(game.score), True, (0, 0, 0))
        screen.blit(text, [0, 0])

        if game.state == "gameover":
            font_game_over = pygame.font.SysFont('Calibri', 65, True, False)
            text_game_over = font_game_over.render("Game Over", True, (255, 0, 0))
            screen.blit(text_game_over, [40, 200])

        pygame.display.flip()
        clock.tick(fps)

    pygame.quit()


#遊戲四
def tictactoe():
    import pygame
    import random

    pygame.init()
    WIDTH, HEIGHT = 300, 300
    LINE_WIDTH = 5
    BOARD_ROWS = 3
    BOARD_COLS = 3
    SQUARE_SIZE = WIDTH // BOARD_COLS

    WHITE = (255, 255, 255)
    LINE_COLOR = (0, 0, 0)
    CIRCLE_COLOR = (0, 0, 255)
    CROSS_COLOR = (255, 0, 0)

    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Tic Tac Toe")

    board = [['' for _ in range(BOARD_COLS)] for _ in range(BOARD_ROWS)]
    state = {
        'game_over': False,
        'winner_text': '',
        'player': 'X'
    }

    def draw_board():
        screen.fill(WHITE)
        for row in range(1, BOARD_ROWS):
            pygame.draw.line(screen, LINE_COLOR, (0, row * SQUARE_SIZE), (WIDTH, row * SQUARE_SIZE), LINE_WIDTH)
        for col in range(1, BOARD_COLS):
            pygame.draw.line(screen, LINE_COLOR, (col * SQUARE_SIZE, 0), (col * SQUARE_SIZE, HEIGHT), LINE_WIDTH)

    def draw_marks():
        for row in range(BOARD_ROWS):
            for col in range(BOARD_COLS):
                mark = board[row][col]
                if mark == 'O':
                    pygame.draw.circle(screen, CIRCLE_COLOR,
                                       (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2),
                                       SQUARE_SIZE // 3, LINE_WIDTH)
                elif mark == 'X':
                    start_pos = (col * SQUARE_SIZE + 20, row * SQUARE_SIZE + 20)
                    end_pos = (col * SQUARE_SIZE + SQUARE_SIZE - 20, row * SQUARE_SIZE + SQUARE_SIZE - 20)
                    pygame.draw.line(screen, CROSS_COLOR, start_pos, end_pos, LINE_WIDTH)
                    pygame.draw.line(screen, CROSS_COLOR, (start_pos[0], end_pos[1]), (end_pos[0], start_pos[1]), LINE_WIDTH)

    def check_winner(mark):
        for row in board:
            if all(cell == mark for cell in row):
                return True
        for col in range(BOARD_COLS):
            if all(board[row][col] == mark for row in range(BOARD_ROWS)):
                return True
        if all(board[i][i] == mark for i in range(BOARD_ROWS)):
            return True
        if all(board[i][BOARD_ROWS - i - 1] == mark for i in range(BOARD_ROWS)):
            return True
        return False

    def ai_move():
        empty = [(r, c) for r in range(3) for c in range(3) if board[r][c] == '']
        if empty:
            row, col = random.choice(empty)
            board[row][col] = 'O'

    def show_winner(text):
        font = pygame.font.SysFont(None, 48)
        text_surface = font.render(text, True, (0, 128, 0))
        text_rect = text_surface.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text_surface, text_rect)

    running = True
    while running:
        screen.fill(WHITE)
        if state['game_over']:
            show_winner(state['winner_text'])
            pygame.display.update()
            pygame.time.delay(2000)
            running = False
            continue
        else:
            draw_board()
            draw_marks()
            pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if not state['game_over'] and event.type == pygame.MOUSEBUTTONDOWN and state['player'] == 'X':
                x, y = event.pos
                row = y // SQUARE_SIZE
                col = x // SQUARE_SIZE
                if board[row][col] == '':
                    board[row][col] = state['player']
                    if check_winner(state['player']):
                        state['winner_text'] = "player wins!"
                        state['game_over'] = True
                    else:
                        state['player'] = 'O'

        if not state['game_over'] and state['player'] == 'O':
            pygame.time.delay(500)
            ai_move()
            if check_winner('O'):
                state['winner_text'] = "computer wins!"
                state['game_over'] = True
            else:
                state['player'] = 'X'

    pygame.quit()


# 主視窗設定
root = tk.Tk()
root.title("小遊戲平台")
root.geometry("400x300")

# 上方分數區
score_label_1 = tk.Label(root, text="", font=("Arial", 14))
score_label_2 = tk.Label(root, text="", font=("Arial", 14))
score_label_1.pack()
score_label_2.pack()
update_scores()

# 中間小遊戲按鈕
tk.Button(root, text="1-100 踩地雷遊戲", command=game_minesweeper).pack(pady=5)
tk.Button(root, text="簡化 Yahtzee 對戰", command=game_yahtzee).pack(pady=5)
tk.Button(root, text="俄羅斯方塊（Pygame）", command=lambda: root.after(100, game_tetris)).pack(pady=5)
ttt_button = tk.Button(root, text="井字遊戲", command=tictactoe)
ttt_button.pack(pady=5)

# 啟動主迴圈
root.mainloop()
